In [53]:
from pypdf import PdfReader
import warnings
warnings.filterwarnings("ignore")
from langchain_text_splitters import RecursiveCharacterTextSplitter
from sentence_transformers import SentenceTransformer
import numpy as np
import chromadb
from chromadb.config import Settings
from sklearn.metrics.pairwise import cosine_similarity
from transformers import AutoTokenizer, AutoModelForCausalLM
import json
import re

In [52]:
pdf_path = "Pensamento_maquina.pdf"

reader = PdfReader(pdf_path)

print(f"Numéro de páginas : {len(reader.pages)}")


Numéro de páginas : 4


In [2]:
full_text = ""

for i, page in enumerate(reader.pages):
    text = page.extract_text()
    full_text += text + "\n"

print("Tamanho total do texto:", len(full_text))

Tamanho total do texto: 9049


In [54]:
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=500,     
    chunk_overlap=100,  
    separators=["\n\n", "\n", ".", " ", ""]
)

chunks = text_splitter.split_text(full_text)

print("Número total de chunks:", len(chunks))

for i, chunk in enumerate(chunks[:3]):
    print("\n====================")
    print(f"CHUNK {i}")
    print("====================")
    print(chunk)


Número total de chunks: 24

CHUNK 0
O Pensamento Humano e o Pensamento das 
Máquinas
Introdução
Desde os primórdios da ﬁlosoﬁa, o ser humano busca compreender a própria mente. 
Perguntas como o que é pensar?, como surge o conhecimento? e o que nos torna conscientes? 
atravessam séculos de reﬂexão ﬁlosóﬁca, cientíﬁca e cultural. Com o avanço da 
tecnologia e, especialmente, com o surgimento da Inteligência Artiﬁcial, essas questões 
ganharam uma nova dimensão: ao criar máquinas capazes de executar tarefas cognitivas,

CHUNK 1
ganharam uma nova dimensão: ao criar máquinas capazes de executar tarefas cognitivas, 
o ser humano passou a se perguntar se estaria, de alguma forma, reproduzindo o próprio 
pensamento.
A relação entre o pensamento humano e o pensamento das máquinas não é apenas 
técnica. Trata-se de uma questão conceitual, ﬁlosóﬁca e ética. Comparar esses dois tipos 
de “pensamento” nos obriga a reﬂetir sobre os limites da tecnologia e, ao mesmo tempo, 
sobre a natureza da mente 

In [55]:
model = SentenceTransformer("all-MiniLM-L6-v2")

print("Gerando embeddings dos chunks...\n")

chunk_embeddings = model.encode(chunks)

print("Número de embeddings:", len(chunk_embeddings))
print("Shape de UM embedding:", chunk_embeddings[0].shape)

print("\nPrimeiros 10 números do embedding do CHUNK 0:")
print(chunk_embeddings[0][:10])


Loading weights: 100%|██████████| 103/103 [00:00<00:00, 407.33it/s, Materializing param=pooler.dense.weight]                             
BertModel LOAD REPORT from: sentence-transformers/all-MiniLM-L6-v2
Key                     | Status     |  | 
------------------------+------------+--+-
embeddings.position_ids | UNEXPECTED |  | 

Notes:
- UNEXPECTED	:can be ignored when loading from different task/architecture; not ok if you expect identical arch.


Gerando embeddings dos chunks...

Número de embeddings: 24
Shape de UM embedding: (384,)

Primeiros 10 números do embedding do CHUNK 0:
[ 0.02200048  0.11847524 -0.00923043  0.01974406 -0.07060358  0.00535173
  0.03218772  0.0715929   0.0200606   0.07255962]


In [ ]:
client = chromadb.Client()

collection = client.create_collection(name="rag_test")

print("Collection criada.")


Collection criada.


In [56]:
query = "Diferença entre o pensamento humano e o pensamento da máquina"

print("\nQUERY:")
print(query)

query_embedding = model.encode([query])

similarities = cosine_similarity(query_embedding, chunk_embeddings)[0]

print("\nSimilaridade com cada chunk:\n")

for i, score in enumerate(similarities):
    print(f"Chunk {i}: {score:.4f}")



QUERY:
Diferença entre o pensamento humano e o pensamento da máquina

Similaridade com cada chunk:

Chunk 0: 0.8478
Chunk 1: 0.8614
Chunk 2: 0.7475
Chunk 3: 0.6331
Chunk 4: 0.6360
Chunk 5: 0.6343
Chunk 6: 0.5915
Chunk 7: 0.8080
Chunk 8: 0.7095
Chunk 9: 0.7348
Chunk 10: 0.7665
Chunk 11: 0.4752
Chunk 12: 0.6275
Chunk 13: 0.8131
Chunk 14: 0.7008
Chunk 15: 0.8631
Chunk 16: 0.8645
Chunk 17: 0.8405
Chunk 18: 0.6097
Chunk 19: 0.7525
Chunk 20: 0.7566
Chunk 21: 0.7933
Chunk 22: 0.8519
Chunk 23: 0.8202


In [63]:
ranking = np.argsort(similarities)[::-1]

print("\nRANKING DOS CHUNKS MAIS RELEVANTES:\n")

for i in ranking[:3]:
    print(f"\nChunk {i} | Similaridade: {similarities[i]:.4f}")
    print(f"{chunks[i][:400]} ...")



RANKING DOS CHUNKS MAIS RELEVANTES:


Chunk 16 | Similaridade: 0.8645
ausência de consciência e signiﬁcado por parte das máquinas. O ser humano pensa a 
partir de um ponto de vista interno, enquanto a máquina opera externamente sobre 
dados.
Humanos atribuem sentido às informações com base em valores, experiências e emoções. 
Máquinas não possuem qualquer relação existencial com o que processam. Elas não 
sabem que existem, não têm medo de errar, nem desejo de acert ...

Chunk 15 | Similaridade: 0.8631
reconhecimento de regularidades, algo que as máquinas fazem de forma extremamente 
eﬁciente.
Essas semelhanças funcionais são suﬁcientes para criar a impressão de que as máquinas 
estão “pensando”, mesmo que o processo subjacente seja radicalmente diferente.
Diferenças fundamentais e limites da analogia
A principal diferença entre o pensamento humano e o pensamento da máquina está na 
ausência de  ...

Chunk 1 | Similaridade: 0.8614
ganharam uma nova dimensão: ao criar máquinas capazes 

In [64]:
ids = [f"chunk_{i}" for i in range(len(chunks))]

collection.add(
    documents=chunks,
    embeddings=chunk_embeddings.tolist(),
    ids=ids
)

print("Chunks inseridos no ChromaDB.")
print("Total armazenado:", len(ids))


Chunks inseridos no ChromaDB.
Total armazenado: 24


In [ ]:
model_name = "Qwen/Qwen2-0.5B-Instruct"

tokenizer_llm = AutoTokenizer.from_pretrained(model_name)
model_llm = AutoModelForCausalLM.from_pretrained(model_name)


c:\Users\lucas\AppData\Local\Programs\Python\Python312\Lib\site-packages\huggingface_hub\file_download.py:130: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\lucas\.cache\huggingface\hub\models--Qwen--Qwen2-0.5B-Instruct. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)
Loading weights: 100%|██████████| 290/290 [00:00<00:00, 373.87it/s, Materializing param=m

In [41]:
def generate_answer(prompt):

    messages = [
        {"role": "system", "content": "Responda sempre em português e seja objetivo."},
        {"role": "user", "content": prompt}
    ]

    text = tokenizer_llm.apply_chat_template(
        messages,
        tokenize=False,
        add_generation_prompt=True
    )

    inputs = tokenizer_llm(
        text,
        return_tensors="pt"
    )

    outputs = model_llm.generate(
        **inputs,
        max_new_tokens=150,
        temperature=0.7,
        top_p=0.9,
        do_sample=True
    )

    response = tokenizer_llm.decode(
        outputs[0][inputs["input_ids"].shape[1]:],
        skip_special_tokens=True
    )

    return response.strip()


In [46]:
def mini_rag(query, n_results=3):

    query_embedding = model.encode([query]).tolist()

    results = collection.query(
        query_embeddings=query_embedding,
        n_results=n_results
    )

    retrieved_chunks = results["documents"][0]

    context = "\n".join(retrieved_chunks[:2])

    prompt = f"""
Use apenas o contexto para responder.

Contexto:
{context}

Pergunta:
{query}
"""

    answer = generate_answer(prompt)

    return {
        "query": query,
        "context": context,
        "answer": answer
    }


In [48]:
generate_answer("Me diga 3 principais diferenças entre o cérebro humano e o processamento das máquinas")

'1. Desempenho: O cérebro humano, que é o maior sistema nervoso do corpo humano, é capaz de realizar uma variedade de tarefas complexas e precisas, como aprender novos conhecimentos, tomar decisões, lembrar-se de informações, responder a perguntas e muitos outros. Em contraste, as máquinas podem ser programadas para realizar tarefas mais simples e específicas, mas não têm o mesmo nível de habilidade cognitiva como o cérebro humano.\n\n2. Recursos de aprendizado: Os processadores humanos são capazes de aprender com base no feedback, enquanto as máquinas não possuem essa capacidade. Isso significa que'

In [ ]:
def extrair_json(texto):
    match = re.search(r"\{.*\}", texto, re.DOTALL)
    if match:
        try:
            return json.loads(match.group())
        except:
            return None
    return None


def avaliar_relevancia(pergunta, resposta):

    prompt = f"""
Avalie de 0 a 10 o quanto a resposta responde corretamente a pergunta.

Pergunta:
{pergunta}

Resposta:
{resposta}

Retorne apenas JSON:
{{"relevancia": numero}}
"""

    result = generate_answer(prompt)
    return extrair_json(result)



def avaliar_fidelidade(contexto, resposta):

    prompt = f"""
Avalie de 0 a 10 se a resposta usa apenas informações do contexto.
Se inventar coisas, reduza a nota.

Contexto:
{contexto}

Resposta:
{resposta}

Retorne apenas JSON:
{{"fidelidade": numero}}
"""

    result = generate_answer(prompt)
    return extrair_json(result)


def avaliar_correcao(pergunta, resposta):

    prompt = f"""
Avalie de 0 a 10 se a resposta está factualmente correta
com base em conhecimento geral.

Pergunta:
{pergunta}

Resposta:
{resposta}

Retorne apenas JSON:
{{"correcao": numero}}
"""

    result = generate_answer(prompt)
    return extrair_json(result)



def avaliar_completude(pergunta, resposta):

    prompt = f"""
Avalie de 0 a 10 se a resposta está completa
ou se faltam partes importantes.

Pergunta:
{pergunta}

Resposta:
{resposta}

Retorne apenas JSON:
{{"completude": numero}}
"""

    result = generate_answer(prompt)
    return extrair_json(result)


def avaliar_resposta_rag(pergunta, contexto, resposta):

    return {
        "relevancia": avaliar_relevancia(pergunta, resposta),
        "fidelidade": avaliar_fidelidade(contexto, resposta),
        "correcao": avaliar_correcao(pergunta, resposta),
        "completude": avaliar_completude(pergunta, resposta)
    }


In [51]:
resultado = mini_rag("Explique redes neurais")

avaliacao = avaliar_resposta_rag(
    pergunta=resultado["query"],
    contexto=resultado["context"],
    resposta=resultado["answer"]
)

print(avaliacao)


{'relevancia': {'relevancia': 7}, 'fidelidade': {'fidelidade': 9}, 'correcao': {'correcao': 9}, 'completude': {'completude': 9}}


👉 Context Precision

quanto do contexto usado era relevante

👉 Context Recall

o modelo usou o que deveria usar?

👉 Faithfulness Score

resposta baseada no contexto?

👉 Answer Correctness

resposta está certa?